In [1]:
import numpy as np
import pandas as pd
import sklearn.model_selection as ms
import sklearn.naive_bayes as nb
import sklearn.metrics as sm
import jieba

In [2]:
jieba.load_userdict('dic.txt')
#加载样本，针对每个评论做分词操作
data=pd.read_csv('htl_all.csv')

#下采样 实现样本均衡化
data=data.tail((data['label']==0).sum()*2)

def func(item):
    seg_list=jieba.cut(item)
    return ' '.join(seg_list)

data['review']=data['review'].apply(func)
data

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.917 seconds.
Prefix dict has been built successfully.


,label,review
2879,1,这个 酒店 住 过 了 无数次 了 ， 客观 的 来说 是 个 值得 推荐 的 酒店 ， 曾...
2880,1,这次 和 同事 一起 订 的 房间 ， 我 在 22 楼 ， 房间 明显 没有 同事 10 ...
2881,1,没有 参加 过 评价 ， 但 感觉 很多 酒店 的 评价 不 客观 。 这次 参与 一下 。...
2882,1,地理位置 不错 ， 就是 临街 有点 吵 ， 其他 都 不错
2883,1,"比较 高 的 性价比 , 环境 服务 都 不错 . 68 的 自助餐 也 还 不错 , 每人..."
...,...,...
7760,0,尼斯 酒店 的 几大 特点 ： 噪音 大 、 环境 差 、 配置 低 、 服务 效率 低 。...
7761,0,盐城 来 了 很 多次 ， 第一次 住 盐阜 宾馆 ， 我 的确 很 失望 整个 墙壁 黑咕...
7762,0,看 照片 觉得 还 挺不错 的 ， 又 是 4 星级 的 ， 但 入住 以后 除了 后悔 没...
7763,0,我们 去 盐城 的 时候 那里 的 最低气温 只有 4 度 ， 晚上 冷得 要死 ， 居然 ...


In [3]:
data.head()

,label,review
2879,1,这个 酒店 住 过 了 无数次 了 ， 客观 的 来说 是 个 值得 推荐 的 酒店 ， 曾...
2880,1,这次 和 同事 一起 订 的 房间 ， 我 在 22 楼 ， 房间 明显 没有 同事 10 ...
2881,1,没有 参加 过 评价 ， 但 感觉 很多 酒店 的 评价 不 客观 。 这次 参与 一下 。...
2882,1,地理位置 不错 ， 就是 临街 有点 吵 ， 其他 都 不错
2883,1,"比较 高 的 性价比 , 环境 服务 都 不错 . 68 的 自助餐 也 还 不错 , 每人..."


In [4]:
#针对review这一列做tfidf
import sklearn.feature_extraction.text as ft
cv=ft.CountVectorizer()
bow=cv.fit_transform(data['review'])
tt=ft.TfidfTransformer()
tfidf=tt.fit_transform(bow)
#拆分测试集训练集
train_x,test_x,train_y,test_y=ms.train_test_split(tfidf,data['label'],random_state=7)
#训练朴素贝叶斯模型
model=nb.MultinomialNB()
model.fit(train_x,train_y)
pred_test_y=model.predict(test_x)
print(sm.classification_report(test_y,pred_test_y))

              precision    recall  f1-score   support

           0       0.87      0.86      0.87       640
           1       0.85      0.86      0.86       582

    accuracy                           0.86      1222
   macro avg       0.86      0.86      0.86      1222
weighted avg       0.86      0.86      0.86      1222



In [5]:
data['label'].value_counts()

1    2443
0    2443
Name: label, dtype: int64

In [8]:
#针对一组真实数据，进行情感分析
test_data=['这家酒店不错，吃得好，住得好，玩得好。','这家酒店真垃圾，哪哪都垃圾，再也不来了，谁来谁孙子！',
           '房间还算干净，厕所有一点点味道，其他还凑合。','我不喜欢','不好','下次不来了']
#先分词，然后转成tfidf,在预测
test_data=pd.Series(test_data).apply(func)
bow=cv.transform(test_data)
tfidf=tt.transform(bow)
pred_y=model.predict(tfidf)
pred_y

array([1, 0, 0, 1, 0, 0])

1.通过股民的评论，判断市场风向看涨还是看跌。
2.通过新闻的评论，判断对印度是干他还是冷静
3.通过新闻的评论，判断民众对国家房价的调控政策的赞成与否


In [11]:
test_y != pred_test_y

3829    False
7219     True
6113     True
6572     True
3811    False
        ...  
6676     True
6516    False
3680    False
6001    False
5578    False
Name: label, Length: 1222, dtype: bool

In [13]:
error_mask=(test_y-pred_test_y) !=0
data.loc[error_mask.index][error_mask]

,label,review
7219,0,酒店 真的 像 足 了 2 星级 的 感觉 ， 除了 早餐 不错 ， 从 服务 到 住宿 ，...
6113,0,酒店 比较 偏 了 ， 虽然 是 在 东方 路上 ， 周围 也 没什么 餐馆 可以 吃饭 的
6572,0,总体 感到 如 家 房间 较 干净 的 就是 房间 太小 了 ， 性价比 不高 ， 比 隔壁...
4436,1,我 是 十一 入住 的 ， 酒店 已经 进入 淡季 ， 整个 酒店 只有 一个 餐厅 营业 ...
6230,0,酒店 的 房间 装修 得 飞常好 ， 空间 也 很大 ， 可是 我 非常 不能 理解 的 是...
...,...,...
3329,1,前台 接待 人员 态度 比较 生硬 ， 客户服务 人员 的 态度 比较 好 （ 比较 耐心 ...
3702,1,不错 的 酒店 ！ 就是 入住 时间 得不到 保证 ！ 下午 3 点 以后 到达 仍 说 没...
3985,1,本 以为 458 的 价格 ， 房间 不会 太小 ， 可 进去 以后 ， 还是 小得 让 人...
7080,0,"酒店 说 是 可以 免费 上网 , 但 只是 自带 网线 而言 , 如 要 借用 酒店 提供..."


In [19]:
#输出每个样本的置信概率
pred_prob_y=model.predict_proba(tfidf)
prob_y=pred_prob_y.max(axis=1)
for a,b,c in zip(test_data,pred_y,prob_y):
    print(a,":",b,"->",c)

这家 酒店 不错 ， 吃 得 好 ， 住 得 好 ， 玩得 好 。 : 1 -> 0.7010794325403221
这家 酒店 真 垃圾 ， 哪 哪 都 垃圾 ， 再也 不来 了 ， 谁 来 谁 孙子 ！ : 0 -> 0.8823239238696743
房间 还 算 干净 ， 厕所 有 一点点 味道 ， 其他 还 凑合 。 : 0 -> 0.5482483215931856
我 不 喜欢 : 1 -> 0.7901368822155739
不好 : 0 -> 0.7116833525112581
下次 不来 了 : 0 -> 0.5214596247040061


In [20]:
pred_y

array([1, 0, 0, 1, 0, 0])